In [26]:
import openai
import re
import requests
import sys
from num2words import num2words
from pdfminer.high_level import extract_text
import os
import pandas as pd
import numpy as np

#API_KEY = # SET YOUR OWN API KEY HERE
 

openai.api_type = "azure"
openai.api_base = "<ADD Base URL>"
openai.api_version = "2022-12-01"
openai.api_key = '<ADD OpenAI API key>'

GPT_ENGINE = '<ADD Open AI model deployment name>'

In [143]:
from PyPDF2 import PdfReader
import os
 

filename =f"<ADD file name>"

reader = PdfReader(filename)
pages = reader.pages

article = []
for ind, page in enumerate(pages):
    text = page.extract_text()
    article.append(text)

prompt = " ".join(article)

In [152]:
 
response = openai.Completion.create(
  engine=GPT_ENGINE,
  prompt= prompt,
  temperature=0.9,
  max_tokens=600,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=None)

rst = response['choices'][0]['text']

In [153]:
rst = rst.replace('\n', '')

###  Get File from Blob File

In [177]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

connection_string = "<Add the Connection String>"
blob_svc = BlobServiceClient.from_connection_string(conn_str=connection_string)
container_name = '<Add the Blob container Name>'

container_client = blob_svc.get_container_client(container_name)

In [178]:
blob_name = "<Add the Blob Name>"  ## file name of the pdf 
blob_client = container_client.get_blob_client(blob_name)

download_stream = blob_client.download_blob()

### Read pdf file in Stream and Convert it to String

In [188]:
from io import BytesIO
blob_to_read = BytesIO(download_stream.readall())
import PyPDF2
fileReader = PyPDF2.PdfReader(blob_to_read)
pages = fileReader.pages


article = []
for ind, page in enumerate(pages):
    text = page.extract_text()
    article.append(text)

prompt = " ".join(article)

In [15]:
def recognize_entities(prompt, engine=GPT_ENGINE):
    """Recognize entities in text using OpenAI's text classification API."""
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=0.8,
        max_tokens=100,
    )
    return response.choices[0].text

In [22]:
def splitter(n, s):
    pieces = s.split()
    list_out = [" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n)]
    return list_out

def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

def trim_incomplete(t):
    if t.endswith('.'):
        if not re.search('[a-z]\.$',t):
            t = t[:-1]

    if not t.endswith('.'):
        t = t.rsplit('. ', 1)[:-1]
        t = "".join(t)+'.'
    
    t = t.strip()
    return t